In [2]:
import cv2
import numpy as np

# Görüntüyü yükle
#  görüntüyü okur
img = cv2.imread("resim.png")

# Görüntüyü gri tonlamaya çevirme
# Renkli görüntüyü gri tonlamaya dönüştürür, çünkü kenar algılama daha iyi gri tonlama görüntülerinde çalışır
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Gürültü azaltma (Gaussian Blur)
# Görüntüdeki gürültüyü azaltmak için Gauss bulanıklığı (Gaussian Blur) uygulanır. Bu, kenar algılamanın doğruluğunu artırabilir.
blurred_img = cv2.GaussianBlur(gray, (5, 5), 0)

# Kontrast artırma
# CLAHE (Contrast Limited Adaptive Histogram Equalization) kullanılarak görüntünün kontrastı artırılır.
# Bu, görüntünün daha net hale gelmesini sağlar, özellikle düşük kontrastlı alanlarda faydalıdır.
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced_img = clahe.apply(blurred_img)

# Canny kenar algılama
# Canny kenar algılama algoritması, görüntüdeki kenarları tespit etmek için kullanılır.
canny_edges = cv2.Canny(enhanced_img, 100, 200)

# Sobel kenar algılama
# Sobel operatörü ile x ve y yönlerinde kenarlar tespit edilir. Sobel, kenarların yönünü ve büyüklüğünü çıkarır.
sobelx = cv2.Sobel(enhanced_img, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(enhanced_img, cv2.CV_64F, 0, 1, ksize=3)
# Sobel x ve y yönlerindeki kenarlar karekök alındıktan sonra birleştirilir
sobel_edges = cv2.sqrt(sobelx*2 + sobely*2)

# Prewitt kenar algılama
# Prewitt operatörü, kenarları tespit etmek için kullanılan başka bir yöntemdir.
# Burada Prewitt'in x ve y yönlerindeki çekirdek (kernel) filtreleri tanımlanır.
kernelx = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]], dtype=np.float32)
kernely = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=np.float32)

# Prewitt filtreleri uygulanır
prewittx = cv2.filter2D(enhanced_img, cv2.CV_32F, kernelx)
prewitty = cv2.filter2D(enhanced_img, cv2.CV_32F, kernely)

# Prewitt kenarları karekök alınarak birleştirilir
prewitt_edges = cv2.sqrt(prewittx*2 + prewitty*2)

# Kenarları bul ve konturları çıkar (Canny kenar algılamayı kullanıyoruz)
# Canny kenar algılama ile elde edilen kenarlara göre konturlar (sınırlar) çıkarılır.
contours, _ = cv2.findContours(canny_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Park yerlerini belirlemek için dikdörtgen segmentler çiz
# Elde edilen konturlara dikdörtgenler çizilir, bu dikdörtgenler potansiyel park alanlarını temsil eder.
for cnt in contours:
    # Konturun çevresine dikdörtgen çizilir
    x, y, w, h = cv2.boundingRect(cnt)
    
    # Kenarların belirli bir boyutun üzerinde olup olmadığını kontrol et (minimum boyut ayarı)
    # Yalnızca belirli bir boyuta sahip konturlar dikkate alınır (örneğin, 50x50'den küçük olanlar yok sayılır)
    if w > 50 and h > 50:
        # Dikdörtgen çiz (yeşil: park alanı)
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Boş mu dolu mu kontrolü: Segment içinde yeterli kenar varsa dolu, yoksa boş
        park_area = canny_edges[y:y+h, x:x+w]
        non_zero_pixels = cv2.countNonZero(park_area)
        
        # Kenar yoğunluğu eşiği belirlenir (non-zero pixel sayısı)
        # Eğer segmentin içinde yeterli sayıda kenar (non-zero piksel) varsa, park alanının dolu olduğu kabul edilir
        if non_zero_pixels > 1000:  # Doluluk tespiti için bir eşik değeri
            cv2.putText(img, "DOLU", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            cv2.putText(img, "BOS", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Kenar algılama sonuçlarını göster
# Farklı kenar algılama tekniklerinin (Canny, Sobel, Prewitt) sonuçları ve park yeri tespiti gösterilir.
cv2.imshow('Canny Kenar Algılama', canny_edges)
cv2.imshow('Sobel Kenar Algılama', sobel_edges)
cv2.imshow('Prewitt Kenar Algılama', prewitt_edges)
cv2.imshow('Park Yeri Tespiti', img)

# Klavye tuşuna basılana kadar görüntülerin açık kalmasını sağlar
cv2.waitKey(0)
cv2.destroyAllWindows()